In [1]:
import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy as np

# load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)

print('data loaded')

data loaded


In [2]:
X_train = x_train.reshape(50000, -1)
X_test = x_test.reshape(10000, -1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalization - converting ints to doubles
X_train /= 255
X_test /= 255
X_train.shape, X_test.shape

((50000, 3072), (10000, 3072))

In [3]:
n_inputs = 3072  # Each image now is 3072 long  #
n_hidden1 = 400  # Num of nodes in hidden layer 1
n_hidden2 = 250  # Num of nodes in hidden layer 2     
n_hidden3 = 64   # Num of nodes in hidden layer 3
n_outputs = 10   # number of classes

In [4]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X") 
y = tf.placeholder(tf.int32, shape=(None), name = "y")
training = tf.placeholder_with_default(False, shape=(), name='training')

In [5]:
from functools import partial  

batch_norm_momentum = 0.9
with tf.name_scope("dnn"):
    
    he_init = tf.contrib.layers.xavier_initializer()
    
    # 3 Hidden layers, each using sigmoid activation function

    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
    bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9) 
    # Momentum of 0.9 is best known for optimization.
    bn1Activation = tf.nn.sigmoid(bn1)
    
    hidden2 = tf.layers.dense(bn1Activation, n_hidden2, name = "hidden2")
    bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
    bn2Activation = tf.nn.sigmoid(bn2)

    hidden3 = tf.layers.dense(bn2Activation, n_hidden3, name = "hidden3")
    bn3 = tf.layers.batch_normalization(hidden3, training = training, momentum = 0.9)
    bn3Activation = tf.nn.sigmoid(bn3)
    logits_before_bn = tf.layers.dense(bn3Activation, n_outputs, name = "outputs") 
    

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits =logits_before_bn)
    # labels - vector must provide a single specific index for the true class
    
    loss = tf.reduce_mean(xentropy, name="loss")
    

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.05, epsilon=1e-06, use_locking=True,name='Adam')
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits_before_bn, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [6]:
init = tf.global_variables_initializer()      # initializes the tf.variable/constant/placeholders declared above
saver = tf.train.Saver()                      # helps to save the model and reuse later.

In [7]:
# Now divide the test and train datasets into 3 parts, training set, validation set, and testing set
# Considering the validation set to have 5000 images
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [8]:
n_epochs = 25
batch_size = 6000 

# A function to shuffle the whole data set and select different batches randomly with replacement
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch                         # returns the first batch that was formed

## With sigmoid activation function

In [9]:
# Now lets test the model with the built model on Validation Set

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid.reshape([-1])})
        print(epoch, "Validation accuracy:", accuracy_val)

0 Validation accuracy: 0.1096
1 Validation accuracy: 0.174
2 Validation accuracy: 0.224
3 Validation accuracy: 0.2676
4 Validation accuracy: 0.2788
5 Validation accuracy: 0.3132
6 Validation accuracy: 0.3362
7 Validation accuracy: 0.3272
8 Validation accuracy: 0.3594
9 Validation accuracy: 0.3462
10 Validation accuracy: 0.359
11 Validation accuracy: 0.3652
12 Validation accuracy: 0.3914
13 Validation accuracy: 0.404
14 Validation accuracy: 0.4132
15 Validation accuracy: 0.3912
16 Validation accuracy: 0.421
17 Validation accuracy: 0.4072
18 Validation accuracy: 0.4282
19 Validation accuracy: 0.4194
20 Validation accuracy: 0.3976
21 Validation accuracy: 0.389
22 Validation accuracy: 0.4392
23 Validation accuracy: 0.4518
24 Validation accuracy: 0.439


In [11]:
# test the results on test set

n_epochs = 25
batch_size = 5000

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        if epoch % 2 == 0:
            accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape([-1])})
            print(epoch, "TestSet accuracy:", accuracy_val)

0 TestSet accuracy: 0.1523
2 TestSet accuracy: 0.3288
4 TestSet accuracy: 0.3507
6 TestSet accuracy: 0.3696
8 TestSet accuracy: 0.4016
10 TestSet accuracy: 0.4267
12 TestSet accuracy: 0.4216
14 TestSet accuracy: 0.4354
16 TestSet accuracy: 0.4551
18 TestSet accuracy: 0.4697
20 TestSet accuracy: 0.4344
22 TestSet accuracy: 0.4649
24 TestSet accuracy: 0.4326


## With RELU activation function

In [12]:
bn1Activation = tf.nn.relu(bn1)
bn2Activation = tf.nn.relu(bn2)
bn3Activation = tf.nn.relu(bn3)

In [13]:
# Now lets test the model with the built model on Validation Set

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid.reshape([-1])})
        print(epoch, "Validation accuracy:", accuracy_val)

0 Validation accuracy: 0.151
1 Validation accuracy: 0.2922
2 Validation accuracy: 0.3204
3 Validation accuracy: 0.3392
4 Validation accuracy: 0.3494
5 Validation accuracy: 0.3562
6 Validation accuracy: 0.3514
7 Validation accuracy: 0.3556
8 Validation accuracy: 0.3636
9 Validation accuracy: 0.3884
10 Validation accuracy: 0.3832
11 Validation accuracy: 0.4032
12 Validation accuracy: 0.4268
13 Validation accuracy: 0.4366
14 Validation accuracy: 0.4496
15 Validation accuracy: 0.4066
16 Validation accuracy: 0.4544
17 Validation accuracy: 0.4564
18 Validation accuracy: 0.4342
19 Validation accuracy: 0.4508
20 Validation accuracy: 0.4028
21 Validation accuracy: 0.4016
22 Validation accuracy: 0.4322
23 Validation accuracy: 0.4156
24 Validation accuracy: 0.4146


In [14]:
# test the results on test set

n_epochs = 25
batch_size = 5000

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        if epoch % 2 == 0:
            accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape([-1])})
            print(epoch, "TestSet accuracy:", accuracy_val)

0 TestSet accuracy: 0.138
2 TestSet accuracy: 0.3189
4 TestSet accuracy: 0.3408
6 TestSet accuracy: 0.3824
8 TestSet accuracy: 0.4095
10 TestSet accuracy: 0.3728
12 TestSet accuracy: 0.4519
14 TestSet accuracy: 0.4494
16 TestSet accuracy: 0.4613
18 TestSet accuracy: 0.4218
20 TestSet accuracy: 0.4283
22 TestSet accuracy: 0.4709
24 TestSet accuracy: 0.4551
